In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# ⚠️ replace `your_module_name` with the filename where your code exists

# ----------------------------
# Load image
# ----------------------------
image_path = r"E:\brand_detection\Brand-Detection\Cloth\a2.png"
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError("Image not found")

# Convert to binary
_, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

kernel_size = (5, 5)
iterations = 2
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)

# ----------------------------
# Manual operations
# ----------------------------
manual_dilate = dilate(img, kernel_size, iterations)
manual_close = close(img, kernel_size)

# ----------------------------
# OpenCV operations
# ----------------------------
cv_dilate = cv2.dilate(binary, kernel, iterations=iterations)
cv_close = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

# ----------------------------
# Difference images
# ----------------------------
dilate_diff = cv2.absdiff(manual_dilate, cv_dilate)
close_diff = cv2.absdiff(manual_close, cv_close)

# ----------------------------
# Display results
# ----------------------------
titles = [
    "Binary Input",
    "Manual Dilation",
    "OpenCV Dilation",
    "Dilation Difference",
    "Manual Close",
    "OpenCV Close",
    "Close Difference"
]

images = [
    binary,
    manual_dilate,
    cv_dilate,
    dilate_diff,
    manual_close,
    cv_close,
    close_diff
]

plt.figure(figsize=(14, 8))
for i, (img, title) in enumerate(zip(images, titles)):
    plt.subplot(2, 4, i + 1)
    plt.imshow(img, cmap="gray")
    plt.title(title)
    plt.axis("off")

plt.tight_layout()
plt.show()

# ----------------------------
# Numeric comparison
# ----------------------------
print("Dilation identical:", np.array_equal(manual_dilate, cv_dilate))
print("Close identical:", np.array_equal(manual_close, cv_close))
print("Dilation pixel difference count:", np.count_nonzero(dilate_diff))
print("Close pixel difference count:", np.count_nonzero(close_diff))


KeyboardInterrupt: 

In [3]:
import cv2
from matplotlib import image
def dilate(image, kernel_size=(5, 5), iterations=2):
    """
    Perform binary dilation manually (without cv2.dilate).
    """
    img = image.copy()
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kh, kw = kernel_size
    pad_h, pad_w = kh // 2, kw // 2

    for _ in range(iterations):
        padded = np.pad(
            img,
            ((pad_h, pad_h), (pad_w, pad_w)),
            mode='constant',
            constant_values=0
        )

        dilated = np.zeros_like(img)

        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                region = padded[i:i+kh, j:j+kw]
                if np.any(region == 255):
                    dilated[i, j] = 255

        img = dilated

    return img


def close(image, kernel):
    """implement your solution and documnet this functoin"""
    dilated = dilate(image, kernel_size)
    closed = erode(dilated, kernel_size)
    return closed



# not used currently
def open(image, kernel):
    """implement your solution and documnet this functoin"""
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

def erode(image, kernel_size=(5, 5), iterations=2):
    """
    Perform erosion manually without using cv2.erode.
    """
    img = image.copy()
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kh, kw = kernel_size
    pad_h, pad_w = kh // 2, kw // 2

    for _ in range(iterations):
        padded = np.pad(
            img,
            ((pad_h, pad_h), (pad_w, pad_w)),
            mode='constant',
            constant_values=255
        )

        eroded = np.zeros_like(img)

        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                region = padded[i:i+kh, j:j+kw]
                if np.all(region == 255):
                    eroded[i, j] = 255

        img = eroded

    return img
